In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
import pinecone
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers

c:\Users\joheb.dani\AppData\Local\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "pcsk_3h5pvh_UEkz4nMcwLcdcdmQNkv8XPzWxh7MhtmztTAqMveMEKpNReKYTdC89i4qzPMCZeJ"
PINECONE_API_ENV = "us-east-1"

In [3]:
# Extract data from the PDF files
def load_pdf(data):
    loader = DirectoryLoader(data, 
                    glob="*.pdf", 
                    loader_cls=PyPDFLoader,
                    show_progress=True)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("data/")

100%|██████████| 1/1 [00:34<00:00, 34.40s/it]


In [5]:
#extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 5859


In [9]:
# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings   

In [10]:
# embeddings = download_hugging_face_embeddings()

!pip install sentence-transformers

In [11]:
embeddings = download_hugging_face_embeddings()


C:\Users\joheb.dani\AppData\Local\Temp\ipykernel_17760\3777190553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [12]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hello World!")
print("Length", len(query_result))

Length 384


In [13]:
#query_result

In [15]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

# If the index doesn't exist, create it
if index_name not in [index.name for index in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,  
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

# Now connect to the index
index = pc.Index(index_name)

# For LangChain integration
from langchain_community.vectorstores import Pinecone as PineconeStore

docsearch = PineconeStore.from_texts(
    [t.page_content for t in text_chunks],
    embeddings,
    index_name=index_name
)


PineconeConfigurationError: You haven't specified an Api-Key.

In [14]:
from langchain_community.vectorstores import Pinecone as PineconeStore

# If index already exists, load it
docsearch = PineconeStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

# Now run your query
query = "What are Allergies"
docs = docsearch.similarity_search(query, k=3)
print(docsearch.index.describe_index_stats())


print("Result", docs)

NameError: name 'index_name' is not defined

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [20]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [21]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'docsearch' is not defined

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])